In [4]:
import utils

import random
import os
import pickle
import time
import gc


import pandas as pd
from pandas import read_csv

import sklearn
from sklearn import ensemble
from sklearn.svm import NuSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import (
    StratifiedKFold,
    train_test_split,
    GridSearchCV,
    cross_validate,
)
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn import naive_bayes
from sklearn import gaussian_process

from statistics import mean
import umap
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

# plt.ioff()
# %matplotlib inline

import numpy as np

from tqdm import tqdm

# import wandb
# wandb.init(project="smart_attacker_same_lightbulb", entity="unr-mpl")

In [5]:
dataset, name_of_current_data = utils.get_dataset()
collected = gc.collect()
print("Garbage collector: collected %d objects." % (collected))

Garbage collector: collected 32 objects.


In [6]:
print(f"*** Total samples in {name_of_current_data}: {len(dataset.index)} ***")
for device_name in sorted(dataset["class"].unique()):
    num_samples = len((dataset[dataset["class"] == device_name]).index)
    print(
        f"*** Samples for device: {device_name} in {name_of_current_data}: {num_samples} ({num_samples/dataset.shape[0]}%) ***"
    )

# classes_to_remove = ["light-4", "light-5", "light-6", "light-7", "light-8",]
# for item in classes_to_remove:
#     dataset = remove_class(item, dataset)
#     dataset.dropna(inplace=True)

# Uncomment this line to take only a portion of the data
# dataset = dataset.head(len(dataset.index)//10)

# x is the entire dataframe except for the class column
x = dataset.drop(["class"], axis=1)

# y_original is an unaltered list of all values in the class column
y_original = dataset["class"].values.tolist()

# y is a dataframe of only the class column and the values have been converted to numeric representation
y = dataset["class"]
counter = 0
y_temp = dataset["class"].tolist()
for unique_value in sorted(y.unique()):
    for index, value in enumerate(y):
        if value == unique_value:
            y_temp[index] = counter
    counter += 1
dataset["class"] = y_temp
y = dataset["class"]
labels_numeric = dataset["class"].unique()

x_train, x_test, y_train, y_test = train_test_split(
    x.values, y.values, test_size=0.2, stratify=y.values
)

del dataset
# del x
# del y
del y_original
del y_temp
del labels_numeric
collected = gc.collect()
print("Garbage collector: collected %d objects." % (collected))

print("*** Dataset Loaded ***")

*** Total samples in SimHash-plug-accum_1024-win_5-combo_4-cleaned: 152009 ***
*** Samples for device: plug-1 in SimHash-plug-accum_1024-win_5-combo_4-cleaned: 16415 (0.10798702708392266%) ***
*** Samples for device: plug-2 in SimHash-plug-accum_1024-win_5-combo_4-cleaned: 15192 (0.09994145083514792%) ***
*** Samples for device: plug-3 in SimHash-plug-accum_1024-win_5-combo_4-cleaned: 17811 (0.11717069384049629%) ***
*** Samples for device: plug-4 in SimHash-plug-accum_1024-win_5-combo_4-cleaned: 21993 (0.14468222276312587%) ***
*** Samples for device: plug-5 in SimHash-plug-accum_1024-win_5-combo_4-cleaned: 19964 (0.1313343288884211%) ***
*** Samples for device: plug-6 in SimHash-plug-accum_1024-win_5-combo_4-cleaned: 14779 (0.09722450644369741%) ***
*** Samples for device: plug-7 in SimHash-plug-accum_1024-win_5-combo_4-cleaned: 30078 (0.19786986296864%) ***
*** Samples for device: plug-8 in SimHash-plug-accum_1024-win_5-combo_4-cleaned: 15777 (0.10378990717654876%) ***
Garbage colle

In [7]:
# Spot Check Algorithms
# x = [1000 for i in range(100)]
# x = (* x,)

models = []
# models.append((1, ensemble.HistGradientBoostingClassifier()))
# models.append((2, ensemble.HistGradientBoostingClassifier(l2_regularization=10, max_depth=8, min_samples_leaf=40, scoring="accuracy")))
# models.append((3, ensemble.RandomForestClassifier(n_jobs=-1)))
# models.append((4, sklearn.neighbors.KNeighborsClassifier(n_jobs=-1)))
# models.append((6, sklearn.linear_model.RidgeClassifier()))
# models.append((7, sklearn.linear_model.SGDClassifier(n_jobs=-1)))
# models.append((1, ensemble.StackingClassifier(estimators=[("HistGrad", ensemble.HistGradientBoostingClassifier()), ("RandForest", ensemble.RandomForestClassifier(n_jobs=-1)), ("KNN", sklearn.neighbors.KNeighborsClassifier(n_jobs=-1))], final_estimator=ensemble.HistGradientBoostingClassifier(), cv=7)))
# models.append((2, ensemble.StackingClassifier(estimators=[("HistGrad", ensemble.HistGradientBoostingClassifier()), ("RandForest", ensemble.RandomForestClassifier(n_jobs=-1)), ("KNN", sklearn.neighbors.KNeighborsClassifier(n_jobs=-1)), ("Ridge", sklearn.linear_model.RidgeClassifier()), ("SGD", sklearn.linear_model.SGDClassifier(n_jobs=-1))], final_estimator=ensemble.HistGradientBoostingClassifier(), cv=7)))
models.append(
    (
        3,
        ensemble.StackingClassifier(
            estimators=[
                ("HistGrad1", ensemble.HistGradientBoostingClassifier()),
                (
                    "HistGrad2",
                    ensemble.HistGradientBoostingClassifier(l2_regularization=1),
                ),
                (
                    "HistGrad3",
                    ensemble.HistGradientBoostingClassifier(l2_regularization=5),
                ),
                (
                    "HistGrad4",
                    ensemble.HistGradientBoostingClassifier(l2_regularization=10),
                ),
                (
                    "RandForest1",
                    ensemble.RandomForestClassifier(n_jobs=14, criterion="gini"),
                ),
                (
                    "RandForest2",
                    ensemble.RandomForestClassifier(n_jobs=14, criterion="entropy"),
                ),
                (
                    "RandForest3",
                    ensemble.RandomForestClassifier(n_jobs=14, criterion="log_loss"),
                ),
                ("KNN1", sklearn.neighbors.KNeighborsClassifier(n_jobs=14, p=1)),
                ("KNN2", sklearn.neighbors.KNeighborsClassifier(n_jobs=14, p=2)),
            ],
            final_estimator=ensemble.HistGradientBoostingClassifier(),
            cv=7,
        ),
    )
)


# evaluate each model
for model_name, model in models:
    # print(f"*** Begin Training and Evaluating {model_name} ***")
    start_time = time.time()
    # print(y_train.shape)
    model.fit(x_train, y_train)
    print(f"*** {model_name} Trained ***")

    y_pred = model.predict(x_test)
    # y_probas = model.predict_proba(x_test)
    total_accuracy = accuracy_score(y_test, y_pred)
    total_f1 = f1_score(y_test, y_pred, average="weighted")
    print(f"Accuracy: {total_accuracy}")
    print(f"F1: {total_f1}")

    # metrics.ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
    # plt.savefig(f"../figures/performance/{name_of_current_data}_confMat.png")

    # param_grid = {"learning_rate": [.1, .3, .5, .7, .9, 1], "max_leaf_nodes": [None, 7, 15, 23, 31, 39, 46], "min_samples_leaf": [10, 20, 100, 500, 1000]}
    # param_grid = {"max_leaf_nodes": [None, 7, 15, 23, 31, 39, 46], "min_samples_leaf": [10, 20, 100, 500, 1000]}
    # search = HalvingGridSearchCV(model, param_grid, resource="n_samples", scoring="accuracy", cv=2, aggressive_elimination=True, n_jobs=-1, verbose=1).fit(x_train, y_train)
    # print(search.best_params_)

    # ******************** #
    # Cross Validation
    # ******************** #
    # cross_val_results = cross_validate(model, x.values, y.values, cv=7, scoring=['accuracy', 'balanced_accuracy', 'f1_weighted', 'confusion_matrix'], n_jobs=7)
    # print(f"*** Finished Training and Evaluating {model_name} ***")
    # print(f"Dataset Name: {name_of_current_data}")
    # print(f"Runtime: {time.time() - start_time}")
    # print(f"Accuracy: {mean(cross_val_results['test_accuracy'])}")
    # print(f"Balanced Accuracy: {mean(cross_val_results['test_balanced_accuracy'])}")
    # # print(f"F1: {mean(cross_val_results['test_f1'])}")
    # print(f"Weighted F1: {mean(cross_val_results['test_f1_weighted'])}")

    # wandb.log({f"Total accuracy TSR on {name_of_current_data}": total_accuracy,
    #            "Dataset": name_of_current_data,
    #            "Num Samples": dataset.shape[0]})
    # wandb.log({f"Total precision TSR on {name_of_current_data}": total_precision,
    #            "Dataset": name_of_current_data,
    #            "Num Samples": dataset.shape[0]})
    # wandb.log({f"Total recall TSR on {name_of_current_data}": total_recall,
    #            "Dataset": name_of_current_data,
    #            "Num Samples": dataset.shape[0]})
    # wandb.log({f"Total f1 TSR on {name_of_current_data}": total_f1,
    #            "Dataset": name_of_current_data,
    #            "Num Samples": dataset.shape[0]})

*** 3 Trained ***
Accuracy: 0.8539240839418459
F1: 0.853968752449742


In [7]:
# def draw_umap(data, n_neighbors, min_dist, n_components, metric, title, save_path):
#     umap_reducer = umap.UMAP(
#         n_neighbors=n_neighbors,
#         min_dist=min_dist,
#         n_components=n_components,
#         metric=metric
#     )
#
#     umap_embedding = umap_reducer.fit_transform(data)
#
#     fig = plt.figure(figsize=(5, 5))
#     if n_components == 1:
#         umap_df = pd.DataFrame(umap_embedding, columns=["dim1"])
#         umap_df["class"] = y_train
#
#         ax = fig.add_subplot(111)
#         ax.scatter(umap_df["dim1"].values, range(len(umap_df.index)), c=umap_df["class"].values, s=1)
#     elif n_components == 2:
#         umap_df = pd.DataFrame(umap_embedding, columns=["dim1", "dim2"])
#         umap_df["class"] = y_train
#
#         ax = fig.add_subplot(111)
#         ax.scatter(umap_df["dim1"].values, umap_df["dim2"].values, c=umap_df["class"].values, s=1)
#     else:
#         umap_df = pd.DataFrame(umap_embedding, columns=["dim1", "dim2", "dim3"])
#         umap_df["class"] = y_train
#         ax = fig.add_subplot(111, projection='3d')
#         ax.scatter(umap_df["dim1"].values, umap_df["dim2"].values,umap_df["dim3"].values, c=umap_df["class"].values, s=1)
#
#     plt.title(title, fontsize=8)
#
#     plt.savefig(save_path, dpi=1200)

In [40]:
# # n_neighbors adjusts the UMAP's attention to local structure vs. global relationships
# # min_dist adjusts how close umap is allowed to place points together
# if not os.path.isdir(f"../figures/{name_of_current_data}/"):
#     os.mkdir(f"../figures/{name_of_current_data}/")
#
# num_generations = 2
# for i in tqdm(range(3)):
#     for j in range(num_generations):
#         n_neighbors = 15
#         min_dist = 0.1
#         n_components = i+1
#         metric = "euclidean"
#         # metric = "minkowski"
#
#         title = f"{name_of_current_data}_{n_neighbors}_{min_dist}_{n_components}_{metric}"
#         save_path = f"../figures/{name_of_current_data}/{n_components}d_{j+1}.png"
#         # save_path = f"/home/nthom/Documents/nilsimsa_vis/{n_components}d_{j+1}.png"
#         draw_umap(x_train, n_neighbors, min_dist, n_components, metric, title, save_path)